In [21]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import qutip
import theano
import theano.tensor as T
import QubitNetwork
import utils
from utils import chop

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
net = QubitNetwork.QubitNetwork(
    num_qubits=4,
    interactions='all',
    self_interactions=('all', ['z']),
    system_qubits=[0, 1, 2]
#     J=new_Jvalues
)
net, _ = QubitNetwork.sgd_optimization(
    net=net,
    learning_rate=1,
    n_epochs=200,
    batch_size=10,
    target_gate=qutip.toffoli(),
    training_dataset_size=100,
    test_dataset_size=1000,
    decay_rate=.01,
    saveafter_file='nets/test.pickle'
)

AttributeError: module 'qutip' has no attribute 'toffolib'

Let us see how the constructed gate actually implements the toffoli.
More specifically, the optimization only assures that we are getting a gate of the form
$$ \mathcal U = \tilde{\mathcal U} \otimes P_A + (...)$$
where $P_A$ is the projector over the ancilla state that was used for the training.
This means that in the case under consideration we should have
$$ \mathcal U = \mathcal U_{toffoli} \otimes \begin{pmatrix}1&0\\0&0\end{pmatrix}$$

In [32]:
net = QubitNetwork.load_network_from_file('nets/toffoli.pickle')
gate = qutip.Qobj(net.get_current_gate(), dims=[[2, 2, 2, 2]] * 2)

p00 = qutip.ket2dm(qutip.basis(2, 0))
p11 = qutip.ket2dm(qutip.basis(2, 1))
p01 = qutip.Qobj([[0, 1], [0, 0]])
p10 = qutip.Qobj([[0, 0], [1, 0]])

i3 = qutip.tensor([qutip.qeye(2) for _ in range(3)])
ntlp = utils.normalize_topleft_phase

extracted1 = (qutip.tensor(i3, p00) * gate * qutip.tensor(i3, p00)).ptrace([0, 1, 2])
extracted2 = (qutip.tensor(i3, p11) * gate * qutip.tensor(i3, p11)).ptrace([0, 1, 2])
extracted3 = (qutip.tensor(i3, p01) * gate * qutip.tensor(i3, p01)).ptrace([0, 1, 2])
extracted4 = (qutip.tensor(i3, p10) * gate * qutip.tensor(i3, p10)).ptrace([0, 1, 2])

display(chop(ntlp(chop(extracted1))))
display(ntlp(chop(extracted2)))
display(ntlp(chop(extracted3)))
display(ntlp(chop(extracted4)))

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = False
Qobj data =
[[ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.]]

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = False
Qobj data =
[[  5.83951148e-01+0.j           1.62434082e-01-0.58318838j
   -3.07493013e-01-0.14184159j  -1.09834616e-01+0.38909997j
    6.55944180e-02-0.04760238j   4.27745341e-03+0.01784894j
    5.83608871e-02+0.04454531j   1.93010523e-02-0.04130145j]
 [  1.62437461e-01-0.58319103j   5.31745528e-01+0.27921003j
   -2.16308406e-01+0.34791808j  -2.72841947e-01-0.15085488j
   -1.41142187e-02-0.01473078j  -1.83223806e-02+0.01353968j
   -5.68142758e-02-0.04111847j   7.74963606e-03+0.04290244j]
 [ -4.62114482e-02+0.18397513j   4.58262614e-01+0.16343632j
   -1.08457710e-01+0.1673485j    7.46808372e-01+0.280339j
   -1.00059273e-01-0.1047949j   -4.23474798e-04+0.00411472j
    1.27962085e-01+0.10846802j  -3.67550730e-02-0.02681088j]
 [  4.78207181e-01+0.10814268j  -6.63162531e-02+0.14325996j
    6.68834189e-01+0.41803916j  -1.84049724e-02+0.22339884j
    6.42882289e-03+0.05188583j   3.40797257e-03+0.059132

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]